# VacationPy

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Output File (CSV)
weather_data = "cities_df.csv"
weather_df = pd.read_csv(weather_data)
weather_df

,City ID,City,Country,Max Temp,Lat,Lng,Humidity,Cloudiness,Wind Speed
0,546105,Nikolskoye,RU,35.71,59.7035,30.7861,97,100,11.70
1,3443061,Chui,UY,65.79,-33.6971,-53.4616,75,22,8.32
2,3359638,Walvis Bay,NaN,68.09,-22.9575,14.5053,85,1,6.91
3,368246,Raja,SS,71.62,8.4596,25.6780,22,56,5.01
4,2015306,Tiksi,RU,-12.68,71.6872,128.8694,93,100,13.06
...,...,...,...,...,...,...,...,...,...
554,1562414,Vung Tau,VN,78.31,10.3460,107.0843,81,62,15.66
555,3671387,Puerto Nariño,CO,75.15,-3.7703,-70.3831,99,99,1.79
556,1273992,Chopda,IN,64.60,21.2500,75.3000,42,56,6.35
557,2014624,Udachny,RU,-4.56,66.4167,112.4000,76,100,3.04


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [14]:
# Store latitude and longitude in locations
locations = weather_df[["Lat", "Lng"]]
humidity = weather_df['Humidity']


In [15]:
# Customize the size of the figure
figure_layout = {
    'width': '800px',
    'height': '700px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Create a dataframe that narrows down the cities based on these criteria: max temp between 70 and 80, wind speed less than 10mph, and cloudiness of zero
weather_cri = pd.DataFrame(weather_df.loc[weather_df['Max Temp'].between(70, 80) & (weather_df['Wind Speed'] <= 10) & (weather_df['Cloudiness'] == 0)])
weather_cri.dropna()
weather_cri.head(5)


,City ID,City,Country,Max Temp,Lat,Lng,Humidity,Cloudiness,Wind Speed
104,5380437,Pacific Grove,US,76.30,36.6177,-121.9166,44,0,5.75
119,3435261,Concordia,AR,74.07,-31.3930,-58.0209,36,0,4.88
124,2290836,Boundiali,CI,74.14,9.5217,-6.4869,17,0,2.08
178,4026082,Lázaro Cárdenas,MX,79.32,17.9583,-102.2000,57,0,8.30
188,3433658,Gualeguaychú,AR,74.89,-33.0094,-58.5172,60,0,8.88


In [6]:
weather_cri.reset_index(drop=True, inplace=True)
weather_cri

,City ID,City,Country,Max Temp,Lat,Lng,Humidity,Cloudiness,Wind Speed
0,5380437,Pacific Grove,US,76.30,36.6177,-121.9166,44,0,5.75
1,3435261,Concordia,AR,74.07,-31.3930,-58.0209,36,0,4.88
2,2290836,Boundiali,CI,74.14,9.5217,-6.4869,17,0,2.08
3,4026082,Lázaro Cárdenas,MX,79.32,17.9583,-102.2000,57,0,8.30
4,3433658,Gualeguaychú,AR,74.89,-33.0094,-58.5172,60,0,8.88
5,3483063,Xicoténcatl,MX,73.20,23.0167,-98.8833,44,0,7.07
6,2070998,Geraldton,AU,70.21,-28.7667,114.6000,64,0,9.22
7,3691582,Talara,PE,77.76,-4.5772,-81.2719,69,0,9.22
8,2409914,Bonthe,SL,77.31,7.5264,-12.5050,88,0,4.88
9,1048364,Chokwé,MZ,73.78,-24.5333,32.9833,73,0,3.98


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# change the variable name of the data
hotel_df = weather_cri

# Add hotel name to the data frame
hotel_df['Hotel Name'] = ''
hotel_df

,City ID,City,Country,Max Temp,Lat,Lng,Humidity,Cloudiness,Wind Speed,Hotel Name
0,5380437,Pacific Grove,US,76.30,36.6177,-121.9166,44,0,5.75,
1,3435261,Concordia,AR,74.07,-31.3930,-58.0209,36,0,4.88,
2,2290836,Boundiali,CI,74.14,9.5217,-6.4869,17,0,2.08,
3,4026082,Lázaro Cárdenas,MX,79.32,17.9583,-102.2000,57,0,8.30,
4,3433658,Gualeguaychú,AR,74.89,-33.0094,-58.5172,60,0,8.88,
5,3483063,Xicoténcatl,MX,73.20,23.0167,-98.8833,44,0,7.07,
6,2070998,Geraldton,AU,70.21,-28.7667,114.6000,64,0,9.22,
7,3691582,Talara,PE,77.76,-4.5772,-81.2719,69,0,9.22,
8,2409914,Bonthe,SL,77.31,7.5264,-12.5050,88,0,4.88,
9,1048364,Chokwé,MZ,73.78,-24.5333,32.9833,73,0,3.98,


In [8]:
# Loop through the hotel_df and get the hotel name 
for index, row in hotel_df.iterrows():
    Lat = row['Lat']
    Lng = row['Lng']

    params = {
        "location": f"{Lat}, {Lng}",
        "keyword": "nearest Hotel",
        "type": "Hotel",
        "radius": "5000",
        "key": g_key
    }
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


    response = requests.get(base_url, params=params).json()
    # Print the name and address of the first bike store that appears   
    try:
        hotel_df.loc[index, 'Hotel Name'] = response['results'][0]['name']
    except:
        print(f"No Hotels found")

No Hotels found
No Hotels found
No Hotels found
No Hotels found


In [9]:
hotel_df

,City ID,City,Country,Max Temp,Lat,Lng,Humidity,Cloudiness,Wind Speed,Hotel Name
0,5380437,Pacific Grove,US,76.30,36.6177,-121.9166,44,0,5.75,The Monterey Hotel
1,3435261,Concordia,AR,74.07,-31.3930,-58.0209,36,0,4.88,Palmar Hotel Casino
2,2290836,Boundiali,CI,74.14,9.5217,-6.4869,17,0,2.08,IMAN S HOTEL
3,4026082,Lázaro Cárdenas,MX,79.32,17.9583,-102.2000,57,0,8.30,Baymont by Wyndham Lazaro Cardenas
4,3433658,Gualeguaychú,AR,74.89,-33.0094,-58.5172,60,0,8.88,Hotel Aguaý
5,3483063,Xicoténcatl,MX,73.20,23.0167,-98.8833,44,0,7.07,
6,2070998,Geraldton,AU,70.21,-28.7667,114.6000,64,0,9.22,Ocean Centre Hotel
7,3691582,Talara,PE,77.76,-4.5772,-81.2719,69,0,9.22,Hostal Oro Negro
8,2409914,Bonthe,SL,77.31,7.5264,-12.5050,88,0,4.88,
9,1048364,Chokwé,MZ,73.78,-24.5333,32.9833,73,0,3.98,


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
# Customize the size of the figure
figure_layout = {
    'width': '800px',
    'height': '700px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
markers = gmaps.marker_layer(locations, hotel_info)


# Add layer
fig.add_layer(markers)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))